In [1]:
from pprint import pprint

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3, bias=True)
        self.bn1 = nn.BatchNorm2d(num_features=3)
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=3, bias=False)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        return F.relu(self.conv2(x))

In [3]:
model = Model()
model

Model(
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), bias=False)
)

In [4]:
for param, weight in model.named_parameters():
    print(f"{param:20} - size: {weight.size()}")
    print(weight)
    print("-" * 100)
    print()

conv1.weight         - size: torch.Size([3, 1, 3, 3])
Parameter containing:
tensor([[[[ 0.1403,  0.2459, -0.0607],
          [-0.2462, -0.1182,  0.1726],
          [-0.2423,  0.0874, -0.0659]]],


        [[[ 0.0838, -0.2309, -0.0206],
          [ 0.0691, -0.2268,  0.2027],
          [-0.0167,  0.1790, -0.3173]]],


        [[[-0.2586,  0.2341,  0.2704],
          [ 0.0466,  0.0301,  0.3105],
          [-0.2876,  0.2565,  0.0192]]]], requires_grad=True)
----------------------------------------------------------------------------------------------------

conv1.bias           - size: torch.Size([3])
Parameter containing:
tensor([ 0.2966, -0.1308,  0.2547], requires_grad=True)
----------------------------------------------------------------------------------------------------

bn1.weight           - size: torch.Size([3])
Parameter containing:
tensor([1., 1., 1.], requires_grad=True)
----------------------------------------------------------------------------------------------------

bn1.b

In [5]:
print(model.conv1.weight)
print(model.conv1.bias)

Parameter containing:
tensor([[[[ 0.1403,  0.2459, -0.0607],
          [-0.2462, -0.1182,  0.1726],
          [-0.2423,  0.0874, -0.0659]]],


        [[[ 0.0838, -0.2309, -0.0206],
          [ 0.0691, -0.2268,  0.2027],
          [-0.0167,  0.1790, -0.3173]]],


        [[[-0.2586,  0.2341,  0.2704],
          [ 0.0466,  0.0301,  0.3105],
          [-0.2876,  0.2565,  0.0192]]]], requires_grad=True)
Parameter containing:
tensor([ 0.2966, -0.1308,  0.2547], requires_grad=True)


In [6]:
import os

save_folder = "./runs/"
save_path = os.path.join(save_folder, "best.pth")   # ./runs/best.pth
os.makedirs(save_folder, exist_ok=True)  

torch.save(model.state_dict(), save_path)
print(f"{save_path} 폴더에 모델이 성공적으로 저장되었습니다.")
print(f"해당 폴더의 파일 리스트: {os.listdir(save_folder)}")

./runs/best.pth 폴더에 모델이 성공적으로 저장되었습니다.
해당 폴더의 파일 리스트: ['best.pth']


In [7]:
new_model = Model()
new_model.load_state_dict(torch.load(save_path))
print(f"{save_path} 에서 성공적으로 모델을 load 하였습니다.")

./runs/best.pth 에서 성공적으로 모델을 load 하였습니다.


In [8]:
for (name, trained_weight), (_, saved_weight) in zip(model.named_parameters(), new_model.named_parameters()):
    is_equal = torch.equal(trained_weight, saved_weight)
    print(f"파라미터 {name:15} 에 대하여 trained 모델과 load 된 모델의 값이 같나요? -> {is_equal}")

파라미터 conv1.weight    에 대하여 trained 모델과 load 된 모델의 값이 같나요? -> True
파라미터 conv1.bias      에 대하여 trained 모델과 load 된 모델의 값이 같나요? -> True
파라미터 bn1.weight      에 대하여 trained 모델과 load 된 모델의 값이 같나요? -> True
파라미터 bn1.bias        에 대하여 trained 모델과 load 된 모델의 값이 같나요? -> True
파라미터 conv2.weight    에 대하여 trained 모델과 load 된 모델의 값이 같나요? -> True


In [9]:
for param, weight in model.state_dict().items():
    print(f"파라미터 네임 {param:25} / 사이즈: {weight.size()}")
    print(weight)
    print("-" * 100, end="\n\n")

파라미터 네임 conv1.weight              / 사이즈: torch.Size([3, 1, 3, 3])
tensor([[[[ 0.1403,  0.2459, -0.0607],
          [-0.2462, -0.1182,  0.1726],
          [-0.2423,  0.0874, -0.0659]]],


        [[[ 0.0838, -0.2309, -0.0206],
          [ 0.0691, -0.2268,  0.2027],
          [-0.0167,  0.1790, -0.3173]]],


        [[[-0.2586,  0.2341,  0.2704],
          [ 0.0466,  0.0301,  0.3105],
          [-0.2876,  0.2565,  0.0192]]]])
----------------------------------------------------------------------------------------------------

파라미터 네임 conv1.bias                / 사이즈: torch.Size([3])
tensor([ 0.2966, -0.1308,  0.2547])
----------------------------------------------------------------------------------------------------

파라미터 네임 bn1.weight                / 사이즈: torch.Size([3])
tensor([1., 1., 1.])
----------------------------------------------------------------------------------------------------

파라미터 네임 bn1.bias                  / 사이즈: torch.Size([3])
tensor([0., 0., 0.])
-----------------

In [10]:
from collections import OrderedDict
print(f"model.state_dict() 의 Type : {type(model.state_dict())}")
isinstance(model.state_dict(), OrderedDict)

model.state_dict() 의 Type : <class 'collections.OrderedDict'>


True

In [11]:
pprint([name for (name, param) in model.named_parameters()])  # named_parameters() : returns only parameters
print()
pprint(list(model.state_dict().keys()))                       # state_dict(): retuns both parameters and buffers

['conv1.weight', 'conv1.bias', 'bn1.weight', 'bn1.bias', 'conv2.weight']

['conv1.weight',
 'conv1.bias',
 'bn1.weight',
 'bn1.bias',
 'bn1.running_mean',
 'bn1.running_var',
 'bn1.num_batches_tracked',
 'conv2.weight']


In [12]:
data = torch.randn(2,2, device=torch.device('cpu'))     # CPU 에 새로운 텐서 생성
print(f"데이터 디바이스: {data.device}")

data = torch.randn(2,2, device=torch.device('cuda:0'))  # GPU 0번에 새로운 텐서 생성
print(f"데이터 디바이스: {data.device}")

data = torch.randn(2,2)                                 # device 를 따로 지정하지 않으면 default 로 CPU 에 생성됩니다.
print(f"데이터 디바이스: {data.device}")

데이터 디바이스: cpu
데이터 디바이스: cuda:0
데이터 디바이스: cpu


In [13]:
model.cpu()
for weight in model.parameters():
    print(f"파라미터 디바이스: {weight.device}")

파라미터 디바이스: cpu
파라미터 디바이스: cpu
파라미터 디바이스: cpu
파라미터 디바이스: cpu
파라미터 디바이스: cpu


In [14]:
model.cuda()
for weight in model.parameters():
    print(f"파라미터 디바이스: {weight.device}")

파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0


In [15]:
device_options = ['cpu', 'cuda']
for device_option in device_options:
    device = torch.device(device_option)
    model.to(device)
    
    print(f"파라미터 디바이스를 {device_option} 로 변경")
    for weight in model.parameters():
        print(f"파라미터 디바이스: {weight.device}")
    print()

파라미터 디바이스를 cpu 로 변경
파라미터 디바이스: cpu
파라미터 디바이스: cpu
파라미터 디바이스: cpu
파라미터 디바이스: cpu
파라미터 디바이스: cpu

파라미터 디바이스를 cuda 로 변경
파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0



In [16]:
data1 = torch.randn(2,2, device=torch.device('cpu'))
data2 = torch.randn(2,2, device=torch.device('cpu'))
print(data1 + data2)  # 두 텐서가 같은 device(CPU) 에 있기에 연산이 가능합니다.

tensor([[-0.7219,  0.8320],
        [-0.9980, -1.6041]])


In [17]:
data1 = torch.randn(2,2, device=torch.device('cpu'))
data2 = torch.randn(2,2, device=torch.device('cuda'))
print(data1 + data2)  # 두 텐서가 다른 device(CPU, GPU) 에 있기에 연산이 불가능합니다.

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [18]:
dummy_input = torch.randn(1, 1, 12, 12, device=device)
model.to(device)
output = model(dummy_input)
print(f"모델 output 사이즈: {output.size()}")
print(output)

모델 output 사이즈: torch.Size([1, 5, 8, 8])
tensor([[[[0.0000, 0.0000, 0.0000, 0.2123, 0.0000, 0.0000, 0.0255, 0.0000],
          [0.0000, 0.4466, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0127, 0.0000, 0.0617, 0.0000, 0.3319, 0.0000, 0.4552, 0.0000],
          [0.2504, 0.0000, 0.2043, 0.1321, 0.0665, 0.5582, 0.2167, 0.3308],
          [0.0000, 0.3592, 0.0939, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.1025, 0.0000, 0.0000, 0.4115, 0.0380, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.4421, 0.2344, 0.0882, 0.0000, 0.0000],
          [0.5158, 0.2553, 0.0000, 0.0000, 0.3839, 0.0270, 0.2654, 0.2163]],

         [[0.0000, 0.0000, 0.0960, 0.0025, 0.0000, 0.0203, 0.0000, 0.0000],
          [0.1283, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2087, 0.2166],
          [0.0000, 0.0000, 0.2113, 0.1103, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.3565, 0.0000, 0.0000, 0.0000, 0.4220],
          [0.0000, 0.0811, 0.1923, 0.1342, 0.0

In [19]:
cpu_device = torch.device('cpu')
gpu_device = torch.device('cuda')

# device is same
dummy_input = dummy_input.to(gpu_device)
model.to(gpu_device)
output = model(dummy_input)  # 잘 작동합니다 
print(f"모델 ouput 사이즈: {output.size()}")

모델 ouput 사이즈: torch.Size([1, 5, 8, 8])


In [20]:
dummy_input = dummy_input.to(cpu_device)
model.to(gpu_device)

# device is different
# RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same
output = model(dummy_input)  # 에러 발생
print(f"모델 ouput 사이즈: {output.size()}")

RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _thnn_conv2d_forward

In [21]:
# requires_grad = False
model.requires_grad_(requires_grad=False)
for param, weight in model.named_parameters():
    print(f"파라미터 {param:15} 가 gradient 를 tracking 하나요? -> {weight.requires_grad}")

파라미터 conv1.weight    가 gradient 를 tracking 하나요? -> False
파라미터 conv1.bias      가 gradient 를 tracking 하나요? -> False
파라미터 bn1.weight      가 gradient 를 tracking 하나요? -> False
파라미터 bn1.bias        가 gradient 를 tracking 하나요? -> False
파라미터 conv2.weight    가 gradient 를 tracking 하나요? -> False


In [22]:
# requires_grad = True
model.requires_grad_(requires_grad=True)
for param, weight in model.named_parameters():
    print(f"파라미터 {param:15} 가 gradient 를 tracking 하나요? -> {weight.requires_grad}")

파라미터 conv1.weight    가 gradient 를 tracking 하나요? -> True
파라미터 conv1.bias      가 gradient 를 tracking 하나요? -> True
파라미터 bn1.weight      가 gradient 를 tracking 하나요? -> True
파라미터 bn1.bias        가 gradient 를 tracking 하나요? -> True
파라미터 conv2.weight    가 gradient 를 tracking 하나요? -> True


In [23]:
model.train()  # train mode 로 전환
print(f"model.bn1.training: {model.bn1.training}")

model.bn1.training: True


In [24]:
model.eval()  # eval mode 로 전환
print(f"model.bn1.training: {model.bn1.training}")

model.bn1.training: False


In [25]:
from torchvision.models import alexnet
model = alexnet()
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 